In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.simplefilter('ignore')
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate


class hybrid(object):
    
    def __init__ (self,user_id,ratings_reviews):
        
        self.user_id = user_id
        self.df = pd.read_csv('C:/Users/mauli/Desktop/FinalData.csv')
        self.ratings_reviews = ratings_reviews
        print(ratings_reviews[(ratings_reviews['user_id'] == user_id)][['user_id','book_id', 'rating']])



        self.popular_ratings = self.popular(self.df)
        self.collaborativebase_rating = self.collaborativebase(self.ratings_reviews, self.user_id)
        self.contentbase_rating = self.contentbase(self.df,self.ratings_reviews,self.user_id)
        self.hybrid_final(self.df, self.popular_ratings , self.collaborativebase_rating, self.contentbase_rating, self.user_id)
        


#popular#

    def popular(self,df):


        ar = pd.read_csv('C:/Users/mauli/Desktop/AverageRatings.csv')
        rc = pd.read_csv('C:/Users/mauli/Desktop/RatingsCount.csv')
	
        ar[ar['rating'].notnull()]['rating'] = ar[ar['rating'].notnull()]['rating'].astype('float')
        averages_vote= ar[ar['rating'].notnull()]['rating'] 
        C = averages_vote.mean()


        rc[rc['rating'].notnull()]['rating'] = rc[rc['rating'].notnull()]['rating'].astype('float')
        counts_vote = rc[rc['rating'].notnull()]['rating']
        m = len(counts_vote)

        df['ratings_reviews_count'] = rc['rating']
        df['average_rating'] = ar['rating']

        accomplished = df[(df['ratings_reviews_count'].notnull())][['book_id','title', 'authors', 'ratings_reviews_count', 'average_rating']]

        accomplished['ratings_reviews_count'] = accomplished['ratings_reviews_count'].astype('float')

        accomplished['average_rating'] = accomplished['average_rating'].astype('float')

        accomplished.shape

        def ratingreviews_weighted(x):
            v = x['ratings_reviews_count']
            R = x['average_rating']
            return (v/(v+m) * R) + (m/(m+v) * C)

        accomplished['popular_ratings'] = accomplished.apply(ratingreviews_weighted, axis=1)
        pop = accomplished[['book_id','popular_ratings']]
        print(accomplished.shape)
        print(pop.shape)

        return pop
    ### Collaborative ##

    def collaborativebase(self,ratings_reviews,user_id):

        reader = Reader()
        #ratings_reviews.head()

        ratings_reviews['book_id'] = ratings_reviews['book_id'].astype('int')
        ratings_reviews['user_id'] = ratings_reviews['user_id'].astype('int')
        ratings_reviews['rating'] = ratings_reviews['rating'].astype('int')
        temp_ratings_reviews = ratings_reviews


        data = Dataset.load_from_df(temp_ratings_reviews[['user_id', 'book_id', 'rating']], reader)
        data.split(n_folds=2)

        ## Training the data ##
        svd = SVD()
        evaluate(svd, data, measures=['RMSE', 'MAE'])

        trainset = data.build_full_trainset()

        algo = SVD()
        algo.fit(trainset)

        #svd.train(trainset)
        ## Testing the data ##

        from collections import defaultdict
        testset = trainset.build_anti_testset()
        predicts = algo.test(testset)

        count = 0
     
        for uid, iid, true_r, est, _ in predicts:

             if uid == user_id:
                count = count+1
                temp_ratings_reviews.loc[len(temp_ratings_reviews)+1]= [uid,iid,est]

        cb = temp_ratings_reviews[(temp_ratings_reviews['user_id'] == user_id)][['book_id', 'rating']]
             
        cb = temp_ratings_reviews[(temp_ratings_reviews['user_id'] == user_id)][['book_id', 'rating']]
        print(cb)
        return(cb)


    ##### CONTENT ######

    def contentbase(self,df,ratings_reviews,user_id):       

        df['book_id'] = df['book_id'].astype('int')
        ratings_reviews['book_id'] = ratings_reviews['book_id'].astype('int')
        ratings_reviews['user_id'] = ratings_reviews['user_id'].astype('int')
        ratings_reviews['rating'] = ratings_reviews['rating'].astype('int')
        df['authors'] = df['authors'].str.replace(' ','')
        df['authors'] = df['authors'].str.lower()
        df['authors'] = df['authors'].str.replace(',',' ')

        #print(df.head())

        df['authors'] = df['authors'].apply(lambda x: [x,x])
        #print(df['authors'])

        df['Genres']=df['Genres'].str.split(';')
        #print(df['Genres'])

        df['soup'] = df['authors'] + df['Genres']

        df['soup'] = df['soup'].str.join(' ')


        count = CountVectorizer(analyzer='word',ngram_range=(1,1),min_df=0, stop_words='english')
        count_matrix = count.fit_transform(df['soup'])
        print (count_matrix.shape)

        cos_sim = cosine_similarity(count_matrix, count_matrix)

        def usersprofile_building():
            userProfiles=np.zeros((60001,999))
		#taking only the first 100000 ratings_reviews to build user_profile
            for i in range(0,100000):
                
                u=ratings_reviews.iloc[i]['user_id']
                b=ratings_reviews.iloc[i]['book_id']
		
                userProfiles[u][b-1]=ratings_reviews.iloc[i]['rating']
                
            return userProfiles

        userProfiles=usersprofile_building()

        def _get_similarItems_to_userProfile(person_id):
            #Computes the cosine similarity between the user profile and all item profiles

            user_ratings_reviews = np.empty((999,1))
            cnt=0
            for i in range(0,998):
                book_sim=cos_sim[i]
                user_sim=userProfiles[person_id]
                user_ratings_reviews[i]=(book_sim.dot(user_sim))/sum(cos_sim[i])
            maxval = max(user_ratings_reviews)
            print(maxval)

            for i in range(0,998):
                user_ratings_reviews[i]=((user_ratings_reviews[i]*5.0)/(maxval))

                if(user_ratings_reviews[i]>3):

                    cnt+=1

            return user_ratings_reviews

        contentbase_ratings_reviews = _get_similarItems_to_userProfile(user_id)



        num = df[['book_id']]
        num1 = pd.DataFrame(data=contentbase_ratings_reviews[0:,0:])
        frames = [num, num1]


        contentbase_rating = pd.concat(frames, axis =1,join_axes=[num.index])
        contentbase_rating.columns=['book_id', 'contentbase_rating']

        return(contentbase_rating)

    
    def hybrid_final(self,df, popular_ratings , collaborativebase_rating, contentbase_rating, user_id):

        hyb = df[['book_id']]
        title = df[['book_id','title', 'Genres']]

        hyb = hyb.merge(title,on = 'book_id')
        hyb = hyb.merge(self.collaborativebase_rating,on = 'book_id')
        hyb = hyb.merge(self.popular_ratings, on='book_id')
        hyb = hyb.merge(self.contentbase_rating, on='book_id')

        def ratingreviews_weighted(x):
            v = x['rating']
            R = x['popular_ratings']
            c = x['contentbase_rating']
            return 0.4*v + 0.2*R + 0.4 * c


        hyb['hyb_rating'] = hyb.apply(ratingreviews_weighted, axis=1)
        hyb = hyb.sort_values('hyb_rating', ascending=False).head(999)
        hyb.columns = ['Book ID' , 'Title', 'Genres', 'Collaborative Rating', 'popular Rating' , 'Content Rating', 'Hybrid Rating']

        print(len(hyb['Hybrid Rating']))
        print(hyb)


def new_user():
    print('\n Rating Reviews from books\n')
    print('ID   Author                           Title                                                              Genre\n')
    print('2.   J.K. Rowling, Mary               Harry Potter and the Sorcerer\'s Stone (Harry Potter, #1)      Fantasy;Young-Age')
    print('127. Malcolm Gladwell                 The Tipping Point: How Little Things Can Make a Big Difference Self-Help')
    print('239. Max Brooks                       World War Z: An Oral History of the Zombie War                 Horror;Fiction')
    print('26   Dan Brown                        The Da Vinci Code                                              Thriller;Drama')
    print('84   Michael Crichton                 Jurassic Park (Jurassic Park, #1)                              SciFi;Thriller;Fantasy')
    print('86   John Grisham                     A Time to Kill                                                 Thriller')
    print('966  Scott Turow                      Presumed Innocent                                              Thriller;Crime')
    print('42   Louisa May Alcott                Little Women (Little Women, #1)                                Young-Age;Romance;Drama')
    print('44   Nicholas Sparks                  The Notebook (The Notebook, #1)                                Romance;Drama')
    print('54   Douglas Adams                    The Hitchhiker\'s Guide to the Galaxy                          Fantasy;Fiction')
    print('134  Cassandra Clare                  City of Glass (The Mortal Instruments, #3)                     Kids;Fantasy;Fiction')
    print('399  J.K. Rowling                     The Tales of Beedle the Bard                                               Kids;Fantasy;Fiction')
    print('38   Audrey Niffenegger               The Time Traveler\'s Wife                                                  Romance;SciFi;Fantasy;Domestic')
    print('729  Dan Simmons                      Hyperion (Hyperion Cantos, #1)                                             SciFi')
    print('807  Dave Eggers                      The Circle                                                                 SciFi')
    print('690  Barack Obama                     The Audacity of Hope: Thoughts on Reclaiming the American Dream            Biography')
    print('617  Piper Kerman                     Orange Is the New Black                                                    Biography')
    print('495  Dave Eggers                      A Heartbreaking Work of Staggering Genius                                  Biography')
    print('770  William Shakespeare,Roma Gill    Julius Caesar                                                              History;Classic')
    print('773  William Shakespeare              The Taming of the Shrew                                                    Comedy;Classic')
    print('829  E.M. Forster                     A Room with a View                                                         Classic')
    print('971  Marcus Pfister, J. Alison James  The Rainbow Fish                                                           Kids')
    print('976  Robert Kapilow, Dr. Seuss        Dr. Seuss\'s Green Eggs and Ham: For Soprano, Boy Soprano, and Orchestra   Kids')
    print('627  Jon Scieszka, Lane Smith         The True Story of the 3 Little Pigs                                        Kids;Fiction')
    print('121  Vladimir Nabokov, Craig Raine    Lolita                                                                     Biography;Romance;Comedy')
    print('196  Chuck Palahniuk                  Fight Club                                                                 Comedy;Drama')
    print('444  A.A. Milne, Ernest H. Shepard    Winnie-the-Pooh (Winnie-the-Pooh, #1)                                      Kids;Comedy')
    print('745  Jenny  Lawson                    Lets Pretend This Never Happened: A Mostly True Memoir                     Biography;Comedy')


    
    ratings_reviews = pd.read_csv('C:/Users/mauli/Desktop/ratings.csv')
    ratings_reviews['book_id'] = ratings_reviews['book_id'].astype('int')
    ratings_reviews['user_id'] = ratings_reviews['user_id'].astype('int')
    ratings_reviews['rating'] = ratings_reviews['rating'].astype('int')
    #taking only the first 100000 ratings_reviews
    ratings_reviews=ratings_reviews[1:100000]
    
    user_id = 60000
    rating_count = len(ratings_reviews['user_id'])+1
    
    print(user_id)
    print('\n----------------Welcome User '+str(user_id)+'-------------------')
    print('\nPlease Provide ratings for the 5 Books from the above list.')
    
    for x in range(0,5):
        print("\n")
        bookId=input("BookId:")
       
        rating=input("Rating:")
        
        ratings_reviews.loc[rating_count]= [user_id,bookId,rating]
        rating_count =rating_count+1
        
    h = hybrid(user_id,ratings_reviews)
        

print("------------------------------Welcome to the Book Recommendation Engine---------------------------\n")

user=input("1.New Users Book Recommendation. \n2.Existing Users Book Recommendation.\n")

if user=='1':
    new_user()
    
elif user=='2':
    ratings_reviews = pd.read_csv('C:/Users/mauli/Desktop/ratings.csv')
    ratings_reviews=ratings_reviews[1:100000]
    #taking only the first 100000 ratings_reviews
    userId=int(input("\nPlease Enter User Id: "))
    print('\n----------------Welcome User'+str(userId)+'-------------------')
    h = hybrid(userId,ratings_reviews)
    
else:
    print("Invalid option\n ")




------------------------------Welcome to the Book Recommendation Engine---------------------------

1.New Users Book Recommendation. 
2.Existing Users Book Recommendation.
2

Please Enter User Id: 123

----------------Welcome User123-------------------
       user_id  book_id  rating
1104       123        5       4
1105       123       26       5
1106       123        4       3
1107       123       58       3
1108       123       32       4
1109       123       64       5
1110       123      111       3
1111       123      100       2
1112       123       45       4
10314      123      272       4
10316      123      189       5
12624      123      132       4
15845      123       57       5
16245      123      130       4
22759      123       81       5
25277      123      383       5
29087      123      860       3
30822      123      228       4
33394      123       11       5
36946      123       15       5
80997      123       22       5
(999, 6)
(999, 2)
Evaluating RMSE, MAE of a